# Calculating the Mulliken population

In [Cedillo2012](https://link.springer.com/article/10.1007/s00214-012-1227-6), we find a Mulliken charge of 0.20 on the carbon atom in CO. In this notebook, we will reproduce this result.

In [3]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

Let's start by creating the molecule, and quantizing the Hamiltonian in the associated restricted spin-orbital basis of the AOs.

## Restricted Mulliken populations

In [15]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/CO_mulliken.xyz" , 0)  # '0': Create a neutral molecule.
N = molecule.numberOfElectrons()

spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
S = spin_orbital_basis.quantizeOverlapOperator()

hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)

Afterwards, we should solve the RHF SCF equations.

In [7]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S.parameters())
solver = gqcpy.RHFSCFSolver.DIIS()
objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # This objective makes sure that the RHF parameters represent the canonical RHF MOs.

rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
D = rhf_parameters.calculateScalarBasis1DM()  # The AO density matrix.

The Mulliken population is effectively the expectation value of the Mulliken operator and a density matrix, so we'll have to set up the Mulliken operator.

Since we're interested in determining the Mulliken charge on the carbon atom, we'll prepare a Mulliken partitioning of the spin-orbital basis with basis functions centered on the carbon atom.

In [8]:
mulliken_partitioning = spin_orbital_basis.mullikenPartitioning(lambda shell: shell.nucleus().element() == "C")

Let's also confirm that the supplied selector lambda function has selected the correct indices for the basis functions centered on the carbon atom.

In [9]:
print(mulliken_partitioning.indices())

[0, 1, 2, 3, 4]


Theoretically, the Mulliken operator is the Mulliken-partitioned number/overlap operator. Since GQCP supports partitioning of any one-electron operator, we can use the `.partitioned()` API.

In [10]:
mulliken_operator = S.partitioned(mulliken_partitioning)

What remains is to calculate the expectation value of the Mulliken-partitioned number operator, which yields the Mulliken population. The Mulliken charge can then be calculated straightforwardly.

In [11]:
C_population = mulliken_operator.calculateExpectationValue(D)[0]
print("Mulliken population: ", C_population)

Mulliken population:  5.801066370434233


In [12]:
C_charge = 6 - C_population
print("Mulliken charge: ", C_charge)

Mulliken charge:  0.19893362956576688


## Unrestricted Mulliken populations

GQCP also offers the ability to calculate alpha- and beta- Mulliken populations. Since the previous example would be trivial, let's use CO+ instead of CO.

In [27]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/CO_mulliken.xyz" , +1)  # '+1': Create CO+.
N_alpha = molecule.numberOfElectronPairs() + 1
N_beta = molecule.numberOfElectronPairs()

u_spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
S_unrestricted = u_spin_orbital_basis.quantizeOverlapOperator()

hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)  # We perform UHF calculations with the Hamiltonian in the basis of the restricted atomic spin-orbitals. 

We solve the UHF SCF equations.

In [30]:
environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S.parameters())
solver = gqcpy.UHFSCFSolver.DIIS()

uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
D_unrestricted = uhf_parameters.calculateScalarBasis1DM()  # The AO density matrix.

We now analogously proceed by setting up the Mulliken partitioning scheme, and finally partition the number/overlap operator according to the obtained partitioning scheme.

In [31]:
u_mulliken_partitioning = u_spin_orbital_basis.mullikenPartitioning(lambda shell: shell.nucleus().element() == "C")
u_mulliken_operator = S_unrestricted.partitioned(u_mulliken_partitioning)

Let's first check the total Mulliken population and charge.

In [36]:
C_population_total = u_mulliken_operator.calculateExpectationValue(D_unrestricted)[0]
print("Mulliken population: ", C_population_total)
print("Mulliken charge: ", 6 - C_population_total)

Mulliken population:  5.202288511329851
Mulliken charge:  0.7977114886701493


Finally, let's find out the Mulliken populations for the alpha and beta electrons, separately

In [38]:
C_population_alpha = u_mulliken_operator.alpha().calculateExpectationValue(D_unrestricted.alpha())[0]
C_population_beta = u_mulliken_operator.beta().calculateExpectationValue(D_unrestricted.beta())[0]

print("Mulliken population alpha: ", C_population_alpha)
print("Mulliken population beta: ", C_population_beta)


Mulliken population alpha:  3.58996514195159
Mulliken population beta:  1.6123233693782613
